In [3]:
import pandas as pd
import os
import requests
import json
import gmplot
from IPython.display import JSON
import gmplot
import pickle

In [4]:
# Yelp API pull
url = 'https://api.yelp.com/v3/businesses/search'
YELP_APIKEY=os.environ['YELP_APIKEY']

class BearerAuth(requests.auth.AuthBase):
    def __init__(self, token):
        self.token = token
    def __call__(self, r):
        r.headers["authorization"] = "Bearer " + self.token
        return r 

In [5]:
def get_business(loc,terms):
    params={
        'location':loc,
        'radius':10000,
        'price':1,
        'term':terms,
        'categories':'Food',
        'limit':50,
        'sort_by':'rating'
        }
    response=requests.get(url,params=params,auth=BearerAuth(YELP_APIKEY))
    return response.json()

In [6]:
bby_snack_yelp = get_business('49.225706, -123.001991','snack') #metrotown 
rmd_snack_yelp = get_business('49.156481, -123.143042','snack') #blundell 
van_dt_snack_yelp = get_business('49.285766, -123.130353','snack') #west-end 
van_west_snack_yelp = get_business('49.241093, -123.140106','snack') #Shaughnessy

In [7]:
pickle.dump(bby_snack_yelp,open('bby_snack_yelp.txt','wb'))
pickle.dump(rmd_snack_yelp,open('rmd_snack_yelp.txt','wb'))
pickle.dump(van_dt_snack_yelp,open('van_dt_snack_yelp.txt','wb'))
pickle.dump(van_west_snack_yelp,open('van_west_snack_yelp.txt','wb'))

In [8]:
bby_snack_yelp=pickle.load(open('bby_snack_yelp.txt','rb'))
rmd_snack_yelp=pickle.load(open('rmd_snack_yelp.txt','rb'))
van_dt_snack_yelp=pickle.load(open('van_dt_snack_yelp.txt','rb'))
van_west_snack_yelp=pickle.load(open('van_west_snack_yelp.txt','rb'))

In [9]:
responses=[bby_snack_yelp,rmd_snack_yelp,van_dt_snack_yelp,van_west_snack_yelp]
area=['bby','rmd','van_dt','van_west']

In [10]:
business={
    'name':[],
    'loc':[],
    'rating':[],
    'price':[],
    'area':[],
    'categories':[]
}

for ind in range(len(responses)):
    a=area[ind]
    response=responses[ind]
    for i in range(len(response['businesses'])):
        business['name'].append(response['businesses'][i]['name'])
        business['loc'].append((
            response['businesses'][i]['coordinates']['latitude'],
            response['businesses'][i]['coordinates']['longitude']))
        business['rating'].append(response['businesses'][i]['rating'])
        business['area'].append(a)
        try:
            business['price'].append(response['businesses'][i]['price'])
        except:
            business['price'].append('na')
        try:
            types=response['businesses'][i]['categories']
            res=[]
            for i in range(len(types)):
                res.append(types[i]['title'])
            business['categories'].append(res)
        except:
            business['categories'].append('na')
        
    

In [11]:
df=pd.DataFrame(data=business,columns=business.keys())
df.sort_values(by='name').head(10)

,name,loc,rating,price,area,categories
129,A Bread Affair,"(49.2724508745628, -123.135854241387)",4.0,$,van_dt,"[Bakeries, Sandwiches]"
177,A Bread Affair,"(49.2724508745628, -123.135854241387)",4.0,$,van_west,"[Bakeries, Sandwiches]"
84,AAA Restaurant,"(49.1785598122947, -123.135279300271)",3.5,$,rmd,"[Noodles, Soup, Chinese]"
42,Al-Salam Halal Meat,"(49.2185936, -122.9597244)",4.0,$,bby,"[Butcher, Halal, Meat Shops]"
34,Altmaerker German Sausage House And Deli,"(49.1559451, -122.9130704)",5.0,$,bby,[Delis]
197,Angus T,"(49.2762326068102, -123.119233623147)",4.0,$,van_west,"[Cafes, Bakeries]"
149,Angus T,"(49.2762326068102, -123.119233623147)",4.0,$,van_dt,"[Cafes, Bakeries]"
144,Asia Market,"(49.207102, -123.0998727)",4.5,$,van_dt,[International Grocery]
76,Asia Market,"(49.207102, -123.0998727)",4.5,$,rmd,[International Grocery]
28,Assi Supermarket,"(49.2236199, -122.98278)",4.0,$,bby,[International Grocery]


In [12]:
pd.DataFrame(df['rating'].value_counts()).reset_index().sort_values(by='index')

,index,rating
4,3.0,7
2,3.5,19
0,4.0,109
1,4.5,54
3,5.0,11


In [13]:
def filter_loc_by_rat(rat):
    filtered_df=df[df['rating']==rat]
    loc=zip(*list(filtered_df['loc']))
    return loc

In [14]:
google_apikey=os.environ['GOOGLE_API_KEY']
gmap = gmplot.GoogleMapPlotter(49.2827,-123.1207,11,apikey=google_apikey)

gmap.scatter(*filter_loc_by_rat(5),color='red', marker=True)
gmap.scatter(*filter_loc_by_rat(4.5),color='orange',marker=True)
gmap.scatter(*filter_loc_by_rat(4),color='orange',marker=True)
gmap.scatter(*filter_loc_by_rat(3),color='yellow',marker=True)
gmap.scatter(*filter_loc_by_rat(3.5),color='yellow',marker=True)
gmap.draw('map.html')

In [15]:
google_apikey=os.environ['GOOGLE_API_KEY']
gmap = gmplot.GoogleMapPlotter(49.2827,-123.1207,10,apikey=google_apikey)

gmap.heatmap(*zip(*list(df['loc'])))
gmap.draw('heatmap.html')

In [16]:
df['categories'][0]

['Bakeries', 'Ukrainian']

In [20]:
flavor=['Asian Fusion','Chinese','French','Hawaiian','Japanese','Indonesian','Korean','Latin American',
 'Malaysian','Mexican',
 'Middle Eastern',
 'Modern European','Russian','Singaporean',
 'Taiwanese',
 'Turkish',
 'Ukrainian',
 'Vietnamese',]

In [21]:
flavors=[]
for word in df['categories'][0]:
    if word in flavor:
        flavors.append(word)

In [22]:
flavors

['Ukrainian']

In [23]:
col=[]
for i in range(df['categories'].shape[0]):
    flavors=[]
    for word in df['categories'][i]:
        if word in flavor:
            flavors.append(word)
        col.append(flavors)

In [24]:
df['categories'].shape[0]

200

In [25]:
words=[]
for i in range(df['categories'].shape[0]):
    for w in df['categories'][i]:
        words.append(w)

In [28]:
f['categories'][0]:
    if word in flavor:
        flavors.append(word)

[['Ukrainian'],
 ['Ukrainian'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Chinese'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Latin American'],
 ['Latin American'],
 ['Vietnamese'],
 ['Vietnamese'],
 [],
 [],
 [],
 ['Japanese'],
 ['Japanese'],
 ['Japanese'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Chinese'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Chinese'],
 ['Chinese'],
 ['Chinese'],
 ['Turkish'],
 ['Turkish'],
 [],
 ['Chinese'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Malaysian', 'Singaporean', 'Chinese'],
 ['Malaysian', 'Singaporean', 'Chinese'],
 ['Malaysian', 'Singaporean', 'Chinese'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Japanese', 'Hawaiian'],
 ['Japanese', 'Hawaiian'],
 ['Japanese', 'Hawaiian'],
 ['Chinese'],
 ['Chinese'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
